# Fundamentals of Machine Learning (CSCI-UA.473)

## Lab 5 : Linear Dimensionality Reduction and Clustering Methods

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from palmerpenguins import load_penguins

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import numpy
from plot_lib import plot_data, plot_model, set_default

set_default()

## Part 1 : Linear Dimensionality Reduction - PCA, SVD and LDA

- The transformation of data from a high-dimensional space to a low-dimensional space
- The transformed representation retains as much information about the original representation as possible
- Useful for data visualization, cluster analysis and classification in fields such as signal processing and computer vision

### Principal Component Analysis (PCA): step-by-step

Computational goal: to find **Principal Components** that 


- Are linear combinations of the original ones

- Are uncorrelated with one another

- Are orthogonal in original dimension space

- Capture as much of the original variance in the data as possible

In [ ]:
from IPython.display import Image
Image('pca.png', width=650)

Given some data represented as an $n\times d$ matrix $X$, where $n$ is the number of samples and $d$ is the dimension of data

In [ ]:
mnist = datasets.load_digits()
X = mnist.data
y = mnist.target
print(X.shape, y.shape)

Find the mean $\mu$ ($d$-dimensional vector) of all samples

In [ ]:
mu = np.mean(X, axis=0)
print(mu)

Compute the covariance matrix $C = (X-\mu)^T(X-\mu)$

In [ ]:
C = (X - mu).T @ (X - mu)
print(C)

Compute the $k$ eigenvectors of $C$ (ordered by decreasing eigenvalues)

In [ ]:
eigvals, eigvecs = np.linalg.eig(C)
print(eigvals.shape, eigvecs.shape, eigvals)

Arrange such eigenvectors in a $d \times k$ matrix $E$

In [ ]:
E = np.copy(eigvecs)
print(E.shape)

Compute the projected samples as $P = X \cdot E$

In [ ]:
P = X @ E
np.shape(P)

Plot the first $2$ principal components of $P$

In [ ]:
target_ids = range(len(mnist.target_names))
plt.figure(figsize=(5, 5))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22']
for i, c, label in zip(target_ids, colors, mnist.target_names):
    plt.scatter(P[y == i, 0], P[y == i, 1], c=c, label=label)
plt.legend()
plt.show()

Compute the reconstruction as $\tilde{X} = P \cdot E^T$

In [ ]:
X_recon = P @ E.T
print(X_recon.shape)

Compute the root-mean-square error (RMSE) between the actual versus reconstructed data

In [ ]:
rmse = np.sqrt(np.mean((X - X_recon)**2))
print(rmse)

### Principal Component Analysis (PCA) using scikit-learn

Load the mnist dataset once again:

In [ ]:
mnist = datasets.load_digits()
X = mnist.data
y = mnist.target
print(X.shape, y.shape)

Fit model to data selecting the first 2 principal components:

In [ ]:
pca = PCA(n_components=10, whiten=True)
pca.fit(X)

Compute the transformed data:

In [ ]:
X_pca = pca.transform(X)

Visualize the transformed data:

In [ ]:
target_ids = range(len(mnist.target_names))
plt.figure(figsize=(5, 5))
for i, c, label in zip(target_ids, colors, mnist.target_names):
    plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1], c=c, label=label)
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend()
plt.show()

In [ ]:
pca.n_components

In [ ]:
explained_var = pca.explained_variance_ratio_
print(f'''The explained variance of PC1 is {explained_var[0]:.2%}
The explained variance of PC2 is {explained_var[1]:.2%}
The total explained variance is {explained_var.sum():.2%}''')

In [ ]:
plt.figure(figsize=(10, 6))
for i in range(pca.n_components):
    plt.subplot(2, 5, i + 1)
    plt.imshow(pca.components_[i].reshape(8,8), interpolation='nearest', clim=(-.15, .15), cmap='gray');
    plt.axis('off');
    

### Singular Value Decomposition (SVD)

The singular value decomposition of an $m\times n$ matrix $M$ is a factorization of the form: 

$$M = U {\Sigma } V^{T} $$

where:


- $U$ is an ${\displaystyle m\times m}$ orthogonal rotation matrix, 

- ${\Sigma }$ is an ${\displaystyle m\times n}$ rectangular diagonal matrix of singular values, and

- $V^{T}$ is an $n\times n$ orthogonal rotation matrix.


Use cases for the SVD:

- Optimization and computational linear algebra: computing the pseudoinverse, solving homogeneous systems of linear equations, etc.

- Machine learning and statistics: dimension reduction via the principal component analysis (PCA) algorithm

- Applications: image compression

In [ ]:
from IPython.display import Image
Image('svd_cartoon.png', width=500)

The singular value decomposition can be done with the `linalg.svd()` function from NumPy (note that `np.linalg.eig(A)` works only on square matrices and will give an error for `A`).

In [ ]:
M = np.array([[7, 2], [3, 4], [5, 3]])
U, S, V = np.linalg.svd(M)

In [ ]:
U

In [ ]:
S

In [ ]:
V

### Image Compression via the Singular Value Decomposition

Load image as ${\displaystyle n\times m}$ matrix of [RGB] values and grayscale it so we are left with ${\displaystyle n\times m}$ matrix of pixel intensity values:

In [ ]:
from PIL import Image
img = Image.open('fox.jpeg') # load
imggray = img.convert('LA') # grayscale
imgmat = np.array(list(imggray.getdata(band=0)), float) # convert to numpy array
imgmat.shape = (imggray.size[1], imggray.size[0]) # get handle on dimensions
plt.figure(figsize=(9, 6))
plt.imshow(imgmat, cmap='gray')
plt.show()

Compute the SVD using NumPy :

In [ ]:
U, S, V = np.linalg.svd(imgmat)
print("img: {}; U: {}; S: {}; V: {}".format(imgmat.shape, U.shape, S.shape, V.shape))

Iterate over list of singular values and plot the reconstructed image for each value:

In [ ]:
for i in [5, 10, 25, 50, 100]:
    reconstruct_img = np.matrix(U[:, :i]) * np.diag(S[:i]) * np.matrix(V[:i, :])
    print("img: {}\nU': {}\nS': {}\nV': {}".format(imgmat.shape, U[:, :i].shape, np.diag(S[:i]).shape,V[:i, :].shape))
    plt.figure(figsize=(6, 4))
    plt.imshow(reconstruct_img, cmap='gray')
    title = "n = %s" % i
    plt.title(title)
    plt.show()

### Relation between PCA and SVD
Simply put, the PCA viewpoint requires that one compute the eigenvalues and eigenvectors of the covariance matrix, which is the product $\frac{1}{n}XX^T$, where $X$
is the centered data matrix. Since the covariance matrix is symmetric, the matrix is diagonalizable, and the eigenvectors can be normalized such that they are orthonormal:
$$ \frac{1}{n}XX^T = \frac{1}{n-1}WDW^T $$

On the other hand, applying SVD to the data matrix $X$ as follows:
$$ X = U\Sigma V^T $$

and attempting to construct the covariance matrix from this decomposition gives
$$\frac{1}{n-1} XX^T = \frac{1}{n-1}(U \Sigma V^T) (U \Sigma V^T)^T = \frac{1}{n-1}(U \Sigma V^T)( V\Sigma U^T)$$

and since $V$ is an orthogonal matrix $(V^T V = I)$,
$$ \frac{1}{n-1}XX^T  = \frac{1}{n-1} U\Sigma^2 U^T $$
and the correspondence is easily seen (the square roots of the eigenvalues of $XX^T$ are the singular values of $X$, etc.)

### Visualize the penguins dataset

In [ ]:
penguins = load_penguins().dropna()
X = penguins[['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']]
y = penguins['species']
print(X.shape, y.shape)
# X.head()
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)

X -= X_mean.to_numpy()
X /= X_std.to_numpy()

In [ ]:
class0 = X[y=='Adelie'].to_numpy()
class1 = X[y=='Chinstrap'].to_numpy()
class2 = X[y=='Gentoo'].to_numpy()

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(class0[:, 0], class0[:, 1], class0[:, 2], c='blue')
ax.scatter(class1[:, 0], class1[:, 1], class1[:, 2], c='red')
ax.scatter(class2[:, 0], class2[:, 1], class2[:, 2], c='green')
ax.set_xlabel('Bill Length')
ax.set_ylabel('Bill Depth')
ax.set_zlabel('Flipper Length')

In [ ]:
U, S, Vh = np.linalg.svd(X.T)
print(U.shape, S.shape, Vh.shape)


In [ ]:
U = U[:, :]
print(U.shape, class0.shape)
Y_0 = U.T @ class0.T
Y_1 = U.T @ class1.T
Y_2 = U.T @ class2.T

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(Y_0[0], Y_0[1], Y_0[2], c='blue')
ax.scatter(Y_1[0], Y_1[1], Y_1[2], c='red')
ax.scatter(Y_2[0], Y_2[1], Y_2[2], c='green')

ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')

### Linear Discriminator Analysis
While goal of PCA is to find components that maximize the variance, LDA makes use of the class labels to maximize the ratio of the between-class scatter matrix to the within-class scatter matrix. 

In [ ]:
mnist = datasets.load_digits()
X = mnist.data
y = mnist.target
print(X.shape, y.shape)
lda = LinearDiscriminantAnalysis(n_components=2)
lda.fit(X, y)

In [ ]:
X_lda = lda.transform(X)

In [ ]:
plt.figure(figsize=(5, 5))
for i, c, label in zip(target_ids, colors, mnist.target_names):
    plt.scatter(X_lda[y==i, 0], X_lda[y==i, 1], c=c, label=label)
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend()
plt.show()

### Classification using LDA
Since LDA is a supervised learning setup, it can directly be used as a classification algorithm as demonstrated below.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Loading 28x28 MNIST to show classification performance.
mnist = fetch_openml('mnist_784')
X, y = mnist['data'], mnist['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=2)
X_train_lda = lda.fit_transform(X_train, y_train)

In [ ]:
X_test_lda = lda.transform(X_test)
y_pred = lda.predict(X_test)
accuracy = sum(y_pred == y_test) / len(y_test)
print("Test set accuracy: ", accuracy)

## Part 2 : Clustering - Kmeans and Gaussian Mixtures

In [ ]:
# Import the necessary packages from sci-kit learn
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster import vq # Specifically uesful for K-means clustering
from sklearn import cluster  # Clustering algorithms such as K-means and agglomerative

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans 
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.manifold import MDS #Import the multidimensional scaling module
from scipy.spatial.distance import squareform #Import squareform, which creates a symmetric matrix from a vector
import time
import math
from sklearn import mixture
from scipy.stats import multivariate_normal as normal # Multivariate Gaussian distributions
from mpl_toolkits.axes_grid1 import ImageGrid

### K-means Clustering

We'll start by looking at the sci-kit learn implementation of K-means for a synthetic dataset that has distinct clusters.  The cell below generates a synthetic dataset with 4 well-separated clusters.

In [ ]:
# Generate the synthetic dataset.
X1 = np.random.multivariate_normal(size = 10, mean = np.array([3, 3]), cov = np.identity(2))
X2 = np.random.multivariate_normal(size = 10, mean = np.array([-3, 3]), cov = np.identity(2))
X3 = np.random.multivariate_normal(size = 10, mean = np.array([-3, -3]), cov = np.identity(2))
X4 = np.random.multivariate_normal(size = 10, mean = np.array([3, -3]), cov = np.identity(2))

X = np.vstack([X1,X2,X3,X4])
plt.scatter(X1[:,0], X1[:,1], c = 'b', marker = 's', label = 'Cluster 1')
plt.scatter(X2[:,0], X2[:,1], c = 'r', marker = 'o', label = 'Cluster 2')
plt.scatter(X3[:,0], X3[:,1], c = 'g', marker = 'p', label = 'Cluster 3')
plt.scatter(X4[:,0], X4[:,1], c = 'm', marker = '+', label = 'Cluster 4')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.title(r'Synthetic dataset with 4 clusters')
plt.legend()
plt.show()

### Distortion

A key quantity in helping determine a good number of clusters to use is the distortion
$$
J = \sum_{i=1}^N \|{\bf x}_i - \mu_{C(i)}\|^2
$$
where ${\bf x}_i$ are the data points, $C(i) \in \{1,\ldots,K\}$ is the cluster assignment for ${\bf x}_i$ and $\mu_j$ for $j=1,\ldots,K$ are the centers of the clusters.  Intuitively, the distortion captures the unexplained variation in the dataset after accounting for the clusters.  If $K = N$, then $\mu_{C(i)} = {\bf x}_i$ and the distortion will be 0.  In this case there is a cluster at every data point so intuitively there is no unexplained variation.  However, having a large number of clusters is often not very useful since we will likely be overfitting to noise in the data.  There will often be a certain point where the distortion starts to decrease more slowly.  This is called the "elbow method", which is what we plot below.

In [ ]:
# Array to keep track of the distortions for K=1,...,N.
distortions = np.zeros(len(X))

for k in range(1, len(X) + 1):
    kmeans = cluster.KMeans(k, n_init="auto") # K-means object in sci-kit learn with k clusters.
    kmeans.fit(X)              # This is the line that actually runs the K-means algorithm.
    distortions[k-1] = kmeans.inertia_ # In sci-kit learn the distortion is called the inertia.

# Plot the results.
plt.plot(np.arange(1, len(X)+1, 1), distortions, 'b-x', lw = 2)
plt.xlabel(r'Clusters $k$')
plt.ylabel(r'Distortion')
plt.title(r'Elbow Curve for K-means')
plt.grid()
plt.show()

Indeed we see from the plot above that the distortion decreases rapidly up until $k=4$, which was the true number of clusters for our data.  After this point we begin overfitting to the noise and the distortion will not decrease as much.  A general heuristic is to choose $k$ where the kink, or elbow, in the curve occurs. 

This data was well-separated, however.  Let's see how the distortion behave whenever there is overlap between the clusters.  The cell below generates the new fake dataset with the clusters closer together.

In [ ]:
# Generate the new synthetic dataset.  The only difference from before is that the means are closer together now.
X1 = np.random.multivariate_normal(size = 10, mean = np.array([1, 1]), cov = np.identity(2))
X2 = np.random.multivariate_normal(size = 10, mean = np.array([-1, 1]), cov = np.identity(2))
X3 = np.random.multivariate_normal(size = 10, mean = np.array([-1, -1]), cov = np.identity(2))
X4 = np.random.multivariate_normal(size = 10, mean = np.array([1, -1]), cov = np.identity(2))

X = np.vstack([X1,X2,X3,X4])
plt.scatter(X1[:,0], X1[:,1], c = 'b', marker = 's', label = 'Cluster 1')
plt.scatter(X2[:,0], X2[:,1], c = 'r', marker = 'o', label = 'Cluster 2')
plt.scatter(X3[:,0], X3[:,1], c = 'g', marker = 'p', label = 'Cluster 3')
plt.scatter(X4[:,0], X4[:,1], c = 'm', marker = '+', label = 'Cluster 4')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.title(r'Synthetic dataset with 4 clusters')
plt.legend()
plt.show()

In [ ]:
# Array of the distortions for k=1,...,N.
distortions = np.zeros(len(X))

# Same code as before.
for k in range(2, len(X) - 1):
    kmeans = cluster.KMeans(k, n_init='auto')
    kmeans.fit(X)
    distortions[k-1] = kmeans.inertia_

# Plot the results.
plt.plot(np.arange(1, len(X)+1, 1), distortions, 'b-x', lw = 2)
plt.xlabel(r'Clusters $k$')
plt.ylabel(r'Distortion')
plt.title(r'Elbow Curve for K-means')
plt.grid()
plt.show()

Now there is a more gradual decrease in the distortion and it is not as clear what choice of $k$ one should use.  

### Silhoutte Scores

![](Sscore.png)

In [ ]:
silhouettes = np.zeros(len(X))

# Same code as before.
for k in range(2, len(X) - 1):
    kmeans = cluster.KMeans(k, n_init='auto')
    labels = kmeans.fit_predict(X)
    silhouettes[k-1] = silhouette_score(X, labels)
    
plt.plot(np.arange(1, len(X)+1, 1), silhouettes, 'r-o', lw = 2)
plt.xlabel(r'Clusters $k$')
plt.ylabel(r'Silhouette Average')
plt.title(r'Elbow Curve for K-means')
plt.grid()
plt.show()

### Initializing the Cluster Centers 
One feature of K-means is that it is prone to becoming stuck in local minimum and is therefore sensitive to the initial cluster centers that are chosen.  The scipy implementation `kmeans2` allows for more flexibility in choosing the initial conditions so we also show it here as an alternative to sci-kit's implementation.  Now we see how the distortion changes after each iteration in K-means for different initializations.

In [ ]:
# Plot the distortion vs iteration for three different initial means.

# Helper function to compute the distortion using the data X and the computed cluster centers
# and labels for each point.
def distortion(X, centers, labels):
    N = X.shape[0]
    J = 0
    for i in range(N):
        J += np.linalg.norm(X[i] - centers[labels[i]])**2
    return J

# Only use 10 iterations of K-means.
max_iter = 10
distortions_1 = np.zeros(max_iter)
distortions_2 = np.zeros(max_iter)
distortions_3 = np.zeros(max_iter)

# 3 different initializations.
K = 4 # 4 clusters
np.random.seed(325) # Random seed is only chosen to emphasize difference between initializations.
                    # This line could be removed.
centers1, labels1 = vq.kmeans2(data = X, k = K, iter = 1, minit = '++')      # k-means++ initialization
centers2, labels2 = vq.kmeans2(data = X, k = K, iter = 1, minit = 'random')  # points sampled from a Gaussian
centers3, labels3 = vq.kmeans2(data = X, k = K, iter = 1, minit = 'points')  # points chosen from the dataset

distortions_1[0] = distortion(X, centers1, labels1)
distortions_2[0] = distortion(X, centers2, labels2)
distortions_3[0] = distortion(X, centers3, labels3)

for i in range(1, max_iter):
    # Do 1 iteration of K-means using the cluster centers from the last iteration.
    centers1, labels1 = vq.kmeans2(data = X, k = centers1, iter = 1, minit = 'matrix')
    centers2, labels2 = vq.kmeans2(data = X, k = centers2, iter = 1, minit = 'matrix')
    centers3, labels3 = vq.kmeans2(data = X, k = centers3, iter = 1, minit = 'matrix')
    
    distortions_1[i] = distortion(X, centers1, labels1)
    distortions_2[i] = distortion(X, centers2, labels2)
    distortions_3[i] = distortion(X, centers3, labels3)

# Plot the results
plt.plot(np.arange(1, max_iter + 1), distortions_1, 'r-s', label = '++')
plt.plot(np.arange(1, max_iter + 1), distortions_2, 'b-o', label = 'random')
plt.plot(np.arange(1, max_iter + 1), distortions_3, 'm-x', label = 'points')
plt.xlabel(r'Iteration')
plt.ylabel(r'Distortion')
plt.title(r'Convergence of K-means for different initializations')
plt.legend()
plt.grid()
plt.show()

The fact that all three curves have leveled-off indicates the K-means has converged.  However, the distortion is different meaning we have congevered to different local optima.

### K-Mediods Clustering <a class="anchor" id="second"></a>

The k-medoids algorithm is a clustering algorithm related to the k-means algorithm and the medoidshift algorithm. Both the k-means and k-medoids algorithms are partitional (breaking the dataset up into groups). K-means attempts to minimize the total squared error, while k-medoids minimizes the sum of dissimilarities between points labeled to be in a cluster and a point designated as the center of that cluster. In contrast to the k-means algorithm, k-medoids chooses datapoints as centers ( medoids or exemplars).

K-medoids is also a partitioning technique of clustering that clusters the data set of n objects into k clusters with k known a priori. A useful tool for determining k is the silhouette.

It could be more robust to noise and outliers as compared to k-means because it minimizes a sum of general pairwise dissimilarities instead of a sum of squared Euclidean distances. The possible choice of the dissimilarity function is very rich but in our applet we used the Euclidean distance.

A medoid of a finite dataset is a data point from this set, whose average dissimilarity to all the data points is minimal i.e. it is the most centrally located point in the set.

The most common realisation of k-medoid clustering is the Partitioning Around Medoids (PAM) algorithm and is as follows:

    * Initialize: randomly select k of the n data points as the medoids
    * Assignment step: Associate each data point to the closest medoid.
    * Update step: For each medoid m and each data point o associated to m swap m and o and compute the total cost of the configuration (that is, the average dissimilarity of o to all the data points associated to m). Select the mediod o with the lowest cost of the configuration.

Repeat alternating steps 2 and 3 until there is no change in the assignments. 

In [ ]:
#define distance metric
euclidean = lambda x1,x2: np.sqrt(np.sum((x1-x2)**2,-1))
manhattan = lambda x1,x2: np.sum(np.abs(x1-x2), -1)
hamming = lambda x1,x2: np.sum((x1!=x2),-1)
dist_dict={"euclidean":euclidean, "manhattan":manhattan, "hamming":hamming}
class KMedoids:
    
    def __init__(self, K=5, max_iters=100, dist_fn=euclidean, repeats=10):
        self.K = K
        self.max_iters = max_iters                        #for computing each medoid
        self.dist_fn = dist_fn
        self.repeats = repeats                            #for several runs to compute medoids
    
    def fit(self, x):
        #note that medoids stores a list of K indices 
        n,d = x.shape
        distances = self.dist_fn(x[None,:,:], x[:,None,:])       #distance function for pairwise distance [n, n]
        best_cost = np.inf
        for r in range(self.repeats):
        #we repeat the process of finding medoids for self.repeats time
            medoids = np.random.choice(n, self.K, replace=False)      #randomly choose a list of k distinct indices from 0 to n-1
            for t in range(self.max_iters):
                membership = np.argmin(distances[medoids,:], axis=0)       #assign membership based on distance from the medoids
                new_medoids = medoids.copy()
                cost = 0
                for i in range(self.K):
                    cluster_inds = np.nonzero(membership == i)[0]                                    #returns the indices of points with membership i
                    cluster_dist = np.sum(distances[np.ix_(cluster_inds, cluster_inds)], axis=1)     #pairwise distance between points with membership i and summed over axis 1  
                    cost += np.min(cluster_dist)                                                     #compute the cost  for the 
                    new_medoids[i] = cluster_inds[np.argmin(cluster_dist)]                           #find the index of i-th medoids
                if np.allclose(new_medoids, medoids):
                    #print(f'converged after {t} iterations with the cost {cost}')
                    break
                medoids = new_medoids
            if cost < best_cost:
                best_medoids = medoids
                best_membership = membership
        return best_medoids, best_membership

In [ ]:
## K-Mediods Visualization
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = datasets.make_blobs(
    n_samples=750, centers=centers, cluster_std=0.4, random_state=0
)
kmediods = KMedoids(K=3)
mediods, labels = kmediods.fit(X)
unique_labels = set(labels)
fig = plt.figure(figsize =(12, 10))
colors = [
    plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))
]
for k, col in zip(unique_labels, colors):

    class_member_mask = labels == k

    xy = X[class_member_mask]
    plt.plot(
        xy[:, 0],
        xy[:, 1],
        "o",
        markerfacecolor=tuple(col),
        markeredgecolor="k",
        markersize=6,
    )
plt.plot(
    X[mediods][:, 0],
    X[mediods][:, 1],
    "o",
    markerfacecolor="cyan",
    markeredgecolor="k",
    markersize=6,
)

plt.title("KMedoids clustering. Medoids are represented in cyan.")
plt.show()

In [ ]:
## Checking the representative digits that we get based on the dissimilarity measure used.
x_org, y = datasets.fetch_openml('mnist_784', version=1, return_X_y=True)
def plot_digits(data):
    num_plots = data.shape[0]
    fig = plt.figure(figsize=(num_plots, 10.*num_plots))
    grid = ImageGrid(fig, 111, nrows_ncols=(1, num_plots), axes_pad=0.1)
    for i in range(num_plots):
        grid[i].imshow(data[i].reshape((28,28)))
    plt.show()

In [ ]:
x = x_org.values[:1000]
for distance in dist_dict.keys():
    kmedoid = KMedoids(10, dist_fn=dist_dict[distance], repeats=1000)
    centers, _ = kmedoid.fit(x.reshape(-1, 784))
    print("Distance Function used: ", distance)
    plot_digits(x[centers])

### Why to use KMediods over KMeans

Pros:
1. _K-medoid is more flexible_: First of all, you can use k-medoids with any similarity measure. K-means however, may fail to converge - it really must only be used with distances that are consistent with the mean. So e.g. Absolute Pearson Correlation must not be used with k-means, but it works well with k-medoids.

2. _Robustness of medoid_: Secondly, the medoid as used by k-medoids is roughly comparable to the median (in fact, there also is k-medians, which is like K-means but for Manhattan distance). If you look up literature on the median, you will see plenty of explanations and examples why the median is more robust to outliers than the arithmetic mean. Essentially, these explanations and examples will also hold for the medoid. It is a more robust estimate of a representative point than the mean as used in k-means.

Cons:
1. _k-medoids is much more expensive_: That's the main drawback. Usually, PAM takes much longer to run than k-means. As it involves computing all pairwise distances, it is $\mathcal{O}(n^2*k*i)$; whereas k-means runs in $\mathcal{O}(n*k*i)$ where usually, k times the number of iterations is $k*i << n$.

### Gaussian mixture models and the EM algorithm

One disadvantage of K-means is that the clusters are restricted to be spherical and so it is heavily dependent on the scaling of the features.  A Gaussian Mixture Model (GMMs) has an extra covariance parameter which allows us to represent poorly-scaled (i.e. long ellipses) data. In addition, GMMs also allow for soft clustering where any point could be contributing to multiple clusters. 

Consider the following toy dataset generated from the Gaussian mixture model with parameters
$$
\phi = (0.5,\ 0.5),\quad \mu_0 = \begin{bmatrix}-3\\0\end{bmatrix},\quad \mu_1 = \begin{bmatrix}3\\0\end{bmatrix}
$$
and
$$
\Sigma_0 = \Sigma_1 = \begin{bmatrix}
1 & 0\\
0 & 100
\end{bmatrix}
$$
The dataset is plotted below.

In [ ]:
# Generate synthetic data
mu1 = np.array([-3, 0])
mu2 = np.array([3, 0])
Sigma = np.array([[1, 0], [0, 100]])
X1 = normal.rvs(mean = mu1, cov = Sigma, size = 25)
X2 = normal.rvs(mean = mu2, cov = Sigma, size = 25)

# Merge the data together.
X = np.vstack([X1, X2])

# Plot the data.
plt.scatter(X1[:,0], X1[:,1], c = 'b', marker = 'o', label = 'Cluster 1')
plt.scatter(X2[:,0], X2[:,1], c = 'r', marker = '+', label = 'Cluster 2')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.title(r'Toy Dataset from GMM')
plt.legend()
plt.show()

Now we'll fit a Gaussian mixture model to this data and plot the contours for the covariance as well as the locations of the means.

In [ ]:
# Create the Gaussian mixture model object with 2 components.  The 'tied' argument refers to the fact that we
# specify the covariances of both components to be the same.  This constrains the problem and reduces the 
# number of parameters we need to estimate.  We also provide the initial parameters which we happen to know already.
gmm = mixture.GaussianMixture(n_components = 2, covariance_type = 'tied', tol=1e-3, \
                              means_init = [mu1, mu2], precisions_init = np.linalg.inv(Sigma))

# The fit function uses the EM algorithm.
gmm.fit(X)

# Get the fitted means and covariances.
phi = gmm.weights_
mu = gmm.means_
cov = gmm.covariances_

# Can also get the number of iterations that was needed for convergence.
print("{:d} iterations for EM to converge.".format(gmm.n_iter_))

# Make a contour plot of the data.
xx = np.linspace(-6, 6, 100)
yy = np.linspace(-20, 20, 100)
XX, YY = np.meshgrid(xx, yy)
points = np.asarray([np.ravel(XX), np.ravel(YY)]).T
Z = phi[0]*normal.pdf(points, mean = mu[0], cov = cov) + phi[1]*normal.pdf(points, mean = mu[1], cov = cov)
ZZ = Z.reshape(XX.shape)

fig, ax = plt.subplots()
ax.contour(XX, YY, ZZ)
plt.scatter(X1[:,0], X1[:,1], c = 'b', marker = 'o', label = 'Cluster 1')
plt.scatter(X2[:,0], X2[:,1], c = 'r', marker = '+', label = 'Cluster 2')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.title(r'Toy Dataset from GMM fitted with EM algorithm')
plt.legend()
plt.axis('tight')
plt.show()
print(mu1, mu2, mu)

## Part 3 : Other clustering algorithms

Both K-means and Gaussian mixture models can only fit elliptical clusters.  Sometimes data may have distinct clusters which do not fit this shape.  The following example is taken from the Sci-kit learn documentation [here](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html).

In [ ]:
import time
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph # kNN is needed for average linkage clustering
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

np.random.seed(0)

# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(n_samples=n_samples,
                             cluster_std=[1.0, 2.5, 0.5],
                             random_state=random_state)

# ============
# Set up cluster parameters
# ============
plt.figure(figsize=(9 * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

plot_num = 1

default_base = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 3,
                'min_samples': 20,
                'xi': 0.05,
                'min_cluster_size': 0.1}

# Parameters for the datasets.
datasets = [
    (noisy_circles, {'damping': .77, 'preference': -240,
                     'quantile': .2, 'n_clusters': 2,
                     'min_samples': 20, 'xi': 0.25}),
    (noisy_moons, {'damping': .75, 'preference': -220, 'n_clusters': 2}),
    (varied, {'eps': .18, 'n_neighbors': 2,
              'min_samples': 5, 'xi': 0.035, 'min_cluster_size': .2}),
    (aniso, {'eps': .15, 'n_neighbors': 2,
             'min_samples': 20, 'xi': 0.1, 'min_cluster_size': .2}),
    (blobs, {}),
    (no_structure, {})]

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create cluster objects
    # ============

    # K-Means algorithm.
    two_means = cluster.KMeans(n_clusters=params['n_clusters'])

    # Average linkage agglomerative clustering.
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average", metric="cityblock",
        n_clusters=params['n_clusters'], connectivity=connectivity)

    # Gaussian mixture model with EM algorithm.
    gmm = mixture.GaussianMixture(
        n_components=params['n_clusters'], covariance_type='full')

    clustering_algorithms = (
        ('KMeans', two_means),
        ('GaussianMixture', gmm),
        ('AgglomerativeClustering', average_linkage)
    )

    for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15,
                 horizontalalignment='right')
        plot_num += 1
plt.show()